In [2]:
import rioxarray
import pandas as pd

# from data module import data processor
import deepsensor.torch
from deepsensor.data import TaskLoader
from deepsensor.data import DataProcessor, construct_circ_time_ds
from deepsensor.data.sources import (
    get_ghcnd_station_data,
    get_era5_reanalysis_data,
    get_earthenv_auxiliary_data,
    get_gldas_land_mask,
)

/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  / np.math.factorial(order)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (D

In [3]:
# Global variables
data_range = ("2015-06-25", "2015-06-30")
extent = "europe"

# Setting cache directory
cache_dir = ".datacache"

## Load data 1: Off-the-gridd station data

In [4]:
data_range = ("2015-06-25", "2015-06-30")
extent = "europe"
# Setting cache directory
cache_dir = ".datacache"

# Station data (off-grid) as pd dataframe
# temp average and precipitation
station_var_IDs = ["TAVG", "PRCP"]

station_raw_df = get_ghcnd_station_data(
    station_var_IDs, extent, date_range=data_range, cache=True, cache_dir=cache_dir
)

In [5]:
station_raw_df = get_ghcnd_station_data(
    station_var_IDs, extent, date_range=data_range, cache=True, cache_dir=cache_dir
)

In [6]:
# 4 variables multi-index
station_raw_df.index

MultiIndex([('2015-06-25',  35.017,   -1.45, 'AGM00060531'),
            ('2015-06-25',    35.1,   -1.85, 'AGE00147716'),
            ('2015-06-25',  35.117,   36.75, 'SYM00040030'),
            ('2015-06-25',  35.167,   2.317, 'AGM00060514'),
            ('2015-06-25',    35.2,  -0.617, 'AGM00060520'),
            ('2015-06-25',  35.208,   0.147, 'AGM00060507'),
            ('2015-06-25', 35.2778, -2.9553, 'SP000060338'),
            ('2015-06-25',  35.333,   4.206, 'AGM00060515'),
            ('2015-06-25', 35.3331, 25.1831, 'GR000016754'),
            ('2015-06-25',  35.341,   1.463, 'AGM00060511'),
            ...
            ('2015-06-30', 69.6767, 18.9131, 'NOE00134898'),
            ('2015-06-30', 69.7556, 27.0119, 'FIE00146698'),
            ('2015-06-30', 69.7889, 20.9553, 'NOE00134922'),
            ('2015-06-30', 69.8333, 21.8842, 'NOE00134934'),
            ('2015-06-30',   69.93,   31.98, 'RSM00022003'),
            ('2015-06-30',  45.933,     7.7, 'ITM00016052'),
        

## Load data 2: Gridded ERA data

In [7]:
era5_var_IDs = ["2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind"]
era5_raw_ds = get_era5_reanalysis_data(
    era5_var_IDs, extent, date_range = data_range, cache = True, cache_dir = cache_dir
)

## Load data 3a: Static (in time) data

- as xarray

In [8]:
auxiliary_var_IDs = ["elevation"]
# auxiliary_var_IDs = ["elevation", "tpi"]

# here in lower res.
da = get_earthenv_auxiliary_data(
    auxiliary_var_IDs, extent, "100KM", cache = True, cache_dir = cache_dir
)

In [9]:
da

<xarray.Dataset>
Dimensions:    (lon: 66, lat: 42)
Coordinates:
  * lon        (lon) float64 -14.58 -13.75 -12.92 -12.08 ... 37.92 38.75 39.58
  * lat        (lat) float64 69.42 68.58 67.75 66.92 ... 37.75 36.92 36.08 35.25
Data variables:
    elevation  (lat, lon) float32 0.0 0.0 0.0 0.0 ... 448.1 594.2 497.0 400.4

## Load data 3b: Static (in time) data land mask

In [11]:
# da: data auxiliary
land_da = get_gldas_land_mask(extent, cache = True, cache_dir = cache_dir)

In [12]:
land_da

<xarray.DataArray 'GLDAS_mask' (lat: 140, lon: 220)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]], dtype=float32)
Coordinates:
  * lon      (lon) float32 -14.88 -14.62 -14.38 -14.12 ... 39.38 39.62 39.88
  * lat      (lat) float32 69.88 69.62 69.38 69.12 ... 35.88 35.62 35.38 35.12

# Initialise Data processor to normalise
- saves parsm to project back
- naming is consitently lat lon

In [14]:
data_processor = DataProcessor(x1_name = "lat", x2_name = "lon")

print(data_processor)

DataProcessor with normalisation params:
{'coords': {'time': {'name': 'time'},
            'x1': {'map': None, 'name': 'lat'},
            'x2': {'map': None, 'name': 'lon'}}}


In [16]:
lowres_auxiliary_var_IDs = ["elevation"]
verbose_download = True

lowres_aux_raw_ds = get_earthenv_auxiliary_data(
    lowres_auxiliary_var_IDs, 
    extent, 
    "100KM", 
    cache = True, 
    cache_dir = cache_dir, 
    verbose = verbose_download)

land_mask_raw_ds = get_gldas_land_mask(
    extent, 
    cache = True, 
    cache_dir = cache_dir, 
    verbose = verbose_download)

In [30]:
era5_ds = data_processor(era5_raw_ds)
station_df = data_processor(station_raw_df)
# elevation, tpi and mask are just min-max normalises
# aux_ds, land_mask_ds = data_processor([da, land_da], method = "min_max")
lowres_aux_ds, land_mask_ds = data_processor([lowres_aux_raw_ds, land_mask_raw_ds], method = "min_max")

In [31]:
lowres_aux_ds

<xarray.Dataset>
Dimensions:    (x2: 66, x1: 42)
Coordinates:
  * x2         (x2) float64 0.007576 0.02273 0.03788 ... 0.9621 0.9773 0.9924
  * x1         (x1) float64 0.6258 0.6106 0.5955 ... 0.03485 0.0197 0.004545
Data variables:
    elevation  (x1, x2) float32 -0.9996 -0.9996 -0.9996 ... -0.5227 -0.6153

In [13]:
era5_ds

<xarray.Dataset>
Dimensions:                  (time: 6, x1: 141, x2: 221)
Coordinates:
  * x1                       (x1) float32 0.6364 0.6318 0.6273 ... 0.004545 0.0
  * x2                       (x2) float32 0.0 0.004545 0.009091 ... 0.9955 1.0
  * time                     (time) datetime64[ns] 2015-06-25 ... 2015-06-30
Data variables:
    2m_temperature           (time, x1, x2) float32 -2.652 -2.635 ... 2.322
    10m_u_component_of_wind  (time, x1, x2) float32 1.987 1.985 ... 1.572 1.529
    10m_v_component_of_wind  (time, x1, x2) float32 1.054 1.018 ... -0.724

In [26]:
land_mask_ds

<xarray.DataArray 'GLDAS_mask' (x1: 140, x2: 220)>
array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.]], dtype=float32)
Coordinates:
  * x2       (x2) float32 0.002273 0.006818 0.01136 ... 0.9886 0.9932 0.9977
  * x1       (x1) float32 0.6341 0.6295 0.625 ... 0.01136 0.006818 0.002273

In [33]:
dates = pd.date_range(era5_ds.time.values.min(), era5_ds.time.values.max(), freq = "D")
# dates.freq
print(dates)
# dtype='datetime64[ns]', freq='D'

# Circular encoding
doy_ds = construct_circ_time_ds(dates, freq = "D")

# Add 2D circular day of year variable to land mask context set
lowres_aux_ds["cos_D"] = doy_ds["cos_D"]
lowres_aux_ds["sin_D"] = doy_ds["sin_D"]

DatetimeIndex(['2015-06-25', '2015-06-26', '2015-06-27', '2015-06-28',
               '2015-06-29', '2015-06-30'],
              dtype='datetime64[ns]', freq='D')


In [35]:
doy_ds

<xarray.Dataset>
Dimensions:  (time: 6)
Coordinates:
  * time     (time) datetime64[ns] 2015-06-25 2015-06-26 ... 2015-06-30
Data variables:
    cos_D    (time) float64 -0.9935 -0.9953 -0.9968 -0.9981 -0.999 -0.9996
    sin_D    (time) float64 0.1137 0.09661 0.07948 0.06232 0.04514 0.02795

In [34]:
lowres_aux_ds

<xarray.Dataset>
Dimensions:    (x2: 66, x1: 42, time: 6)
Coordinates:
  * x2         (x2) float64 0.007576 0.02273 0.03788 ... 0.9621 0.9773 0.9924
  * x1         (x1) float64 0.6258 0.6106 0.5955 ... 0.03485 0.0197 0.004545
  * time       (time) datetime64[ns] 2015-06-25 2015-06-26 ... 2015-06-30
Data variables:
    elevation  (x1, x2) float32 -0.9996 -0.9996 -0.9996 ... -0.5227 -0.6153
    cos_D      (time) float64 -0.9935 -0.9953 -0.9968 -0.9981 -0.999 -0.9996
    sin_D      (time) float64 0.1137 0.09661 0.07948 0.06232 0.04514 0.02795

In [15]:
# Preprocessed land mask
land_mask_ds

<xarray.DataArray 'GLDAS_mask' (x1: 140, x2: 220)>
array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.]], dtype=float32)
Coordinates:
  * x2       (x2) float32 0.002273 0.006818 0.01136 ... 0.9886 0.9932 0.9977
  * x1       (x1) float32 0.6341 0.6295 0.625 ... 0.01136 0.006818 0.002273

In [16]:
land_mask_ds["cos_M"] = 10

In [17]:
land_mask_ds

<xarray.DataArray 'GLDAS_mask' (x1: 140, x2: 220)>
array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.]], dtype=float32)
Coordinates:
  * x2       (x2) float32 0.002273 0.006818 0.01136 ... 0.9886 0.9932 0.9977
  * x1       (x1) float32 0.6341 0.6295 0.625 ... 0.01136 0.006818 0.002273
    cos_M    int64 10

In [27]:
print(data_processor)

DataProcessor with normalisation params:
{'10m_u_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': 0.644899845123291,
                                        'std': 2.8509252071380615}},
 '10m_v_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': -0.19969606399536133,
                                        'std': 3.2448606491088867}},
 '2m_temperature': {'method': 'mean_std',
                    'params': {'mean': 289.39849853515625,
                               'std': 5.538551330566406}},
 'GLDAS_mask': {'method': 'min_max', 'params': {'max': 1.0, 'min': 0.0}},
 'PRCP': {'method': 'mean_std',
          'params': {'mean': 1.1601769356255491, 'std': 4.171802535279706}},
 'TAVG': {'method': 'mean_std',
          'params': {'mean': 19.0613726868119, 'std': 5.188503809362459}},
 'coords': {'time': {'name': 'time'},
            'x1': {'map': (35.0, 90.0), 'name': 'lat'},
            'x2': {'map': (-15.0, 40.0

# Tasks

- time is implictly represented

X_c: context locations
Y_c: context observations
X_t: target locations
Y_t: target observations (training)

Example: 
- two context sets
- first one has 52 locations and 3 context observations
- second one has 112 locations and 1 context observation
- target has 245 locations and 2 target variables at these locations

In [37]:
# from deepsensor.data import TaskLoader
task_loader = TaskLoader(context=[era5_ds, land_mask_ds], target=station_df)
# changed
task = task_loader("2015-06-25", context_sampling=[52, 112], target_sampling=245)

In [40]:
print(task)
# ERA has 3 vars and landmask has just 1 var

time: 2015-06-25 00:00:00
ops: []
X_c: [(2, 52), (2, 112)]
Y_c: [(3, 52), (1, 112)]
X_t: [(2, 245)]
Y_t: [(2, 245)]



In [42]:
task_with_gridded_data = task_loader(
    "2015-06-25", context_sampling=["all", "all"], target_sampling=245
)

In [43]:
print(task_with_gridded_data)

time: 2015-06-25 00:00:00
ops: []
X_c: [((1, 141), (1, 221)), ((1, 140), (1, 220))]
Y_c: [(3, 141, 221), (1, 140, 220)]
X_t: [(2, 245)]
Y_t: [(2, 245)]



# Taskloader/taskdesigns

In [44]:
task_loader = TaskLoader(
    context=[era5_ds, land_mask_ds],
    target=station_df,
    aux_at_targets=aux_ds,
)

print(task_loader)

TaskLoader(2 context sets, 1 target sets)
Context variable IDs: (('2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind'), ('GLDAS_mask',))
Target variable IDs: (('PRCP', 'TAVG'),)
Auxiliary-at-target variable IDs: ('elevation', 'tpi')
